In [ ]:
# %%
# code by Tae Hwan Jung(Jeff Jung) @graykode
# Reference : https://github.com/jadore801120/attention-is-all-you-need-pytorch
#           https://github.com/JayParks/transformer, https://github.com/dhlee347/pytorchic-bert
import math
import re
from random import *
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# sample IsNext and NotNext to be same in small batch size
def make_batch():
    batch = []
    positive = negative = 0
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index= randrange(len(sentences)), randrange(len(sentences)) # 문장 길이 범위내 랜덤 index 설정
        tokens_a, tokens_b= token_list[tokens_a_index], token_list[tokens_b_index] # 입력 문장의 token 중 index로 뽑아내기
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']] # word_dict = PAD + CLS + SEP +  MASK + token
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1) # 0 or1
        # token_a : CLS + tokens_a + SEP
        # token_b : tokens_b + SEP 

        # MASK LM
        # The training data generator chooses 15% of the token positions at random for prediction
        # 훈련 데이터 생성기는 예측을 위해 토큰 위치의 15%를 무작위로 선택합니다
        n_pred =  min(max_pred, max(1, int(round(len(input_ids) * 0.15)))) # 전체 중 15%만 맞출거야!
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                          if token != word_dict['[CLS]'] and token != word_dict['[SEP]']] # input_ids 중 CLS와 SEP token이 아닌 token의 index
        shuffle(cand_maked_pos) # shuffle -> 뒤에 랜덤한 15%를 mask하기 위해 사전에 shuffle
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]: # 15%의 mask 중 -> 사전에 shuffle을 했기 때문에 [:n_pred] = 15% random 추출
            masked_pos.append(pos) # masking 할 token의 index
            masked_tokens.append(input_ids[pos]) # masking 할 token
            if random() < 0.8:  # 80%
                input_ids[pos] = word_dict['[MASK]'] # make mask
            elif random() < 0.5:  # 10%
                index = randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word_dict[number_dict[index]] # replace

        # Zero Paddings -> max_len 사이즈로 zero_padding
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens -> max_pred size 만큼 mask_tokens을 zero_padding
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2: # 위에서 랜덤하게 뽑은 두 문장이 연결되는 문장이고 positive가 batch_size/2라면
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2: # 두 문장이 연결되는 문장이 아니라면
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
    return batch
# Proprecessing Finished

In [ ]:
def get_attn_pad_mask(seq_q, seq_k): # Attention을 구할 때 Padding 부분을 제외하기 위한 Mask
                                     # Padding = False, Tokens = True
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k